# RFC obtaintion (RFC format validation and name cleaning)
**Important:** This report shows the same process presented in 'SLM_report.ipynb', but now Polars is implemented instead of Pandas. See 'SLM_report.ipynb' first.

The goal is to read a csv file with 3 columns ("RFC", "RAZON" and "AÑO"), then to validate RFC format, to clean column "RAZON" by applying Regex techniques to save the cleaned text in column "NOMBRE", and to add column "PERSON" if the RFC corresponds to a "fisica" or "moral" person. Finally, a SLM is implementd to tokenizate and lemmatizate the text in column "NOMBRE" and group by this text similarity. The cleaned DataFrame is saved as "NuevoRFC.csv". This DataFrame can be concated to other DataFrame (optional). Polars is used.

**Remark:** This Jupyter Notebook can be run in Google-Colab.

In [1]:
%pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.5 MB/s eta 0:00:00


In [2]:
# Import libraries
import polars as pl
import re
from typing import Optional
import spacy                # Recommended for Spanish
from rapidfuzz import fuzz

## Read non-processed csv file or create a DataFrame

In [3]:
column_names = ['RFC','RAZON','AÑO']
year = 2025

# Create DataFrame
df = pl.DataFrame({
    'RFC': ['AAA08091161A', 'AAA1002249W5', 'AAA090924HJ4', 'AVM090924HJ4',
            'AAA1002249W6', 'AAA1002249W7', 'BBB08091161A', 'FEM1002249W5',
            'BBV08091161A', 'AAA1002249W8', 'AAA1002249W9', 'AAA1002249W1',
            'AAA1002249W2', 'BBB08091161A', 'FEM1002249W5', 'AAA1002249W5',
            'AVM090924HJ4'],
    'RAZON': ['APOYANDO A ANGELITOS CON AUTISMO, A. C.',
              'LA PASADITA',
              'ARGUELLES, ALVAREZ & ASOCIADOS SA DE CV',
              'ADMINISTRATIVAS APLICACIONES AVM SC',
              'ADAIR ALONSO ARQUITECTOS SA DE CV',
              'Waltmart',
              'BBVA BANCOMER',
              'COCA COLA FEMSA CV',
              'BANCOMER BBVA S.A.',
              'Waltmart de Mexico',
              'apoyando angelitos con autismo ac',
              'MC SA DE CV',
              'S.A. DE C.V. MC',
              'BBVA BANCOMER',
              'FEMSA COCA-COLA',
              'ABARROTES LA PASADITA',
              'AGROINDUSTRIAS APLICACIONES ADMINISTRATIVAS AVM SC',]
})

# Add year
df = df.with_columns(pl.lit(year).alias("AÑO"))

df.head()

RFC,RAZON,AÑO
str,str,i32
"""AAA08091161A""","""APOYANDO A ANGELITOS CON AUTIS…",2025
"""AAA1002249W5""","""LA PASADITA""",2025
"""AAA090924HJ4""","""ARGUELLES, ALVAREZ & ASOCIADOS…",2025
"""AVM090924HJ4""","""ADMINISTRATIVAS APLICACIONES A…",2025
"""AAA1002249W6""","""ADAIR ALONSO ARQUITECTOS SA DE…",2025


In [4]:
"""
csv_file = 'prueba.csv'
column_names = ['RFC','RAZON','AÑO']
df = (
    pl.scan_csv(csv_file, separator=',', has_header=False, new_columns=column_names)
    .filter(pl.col("RFC").is_not_null())    # Filter before processing
    .collect()              # To process the file in chunks
)
df.head(5)
"""

'\ncsv_file = \'prueba.csv\'\ncolumn_names = [\'RFC\',\'RAZON\',\'AÑO\']\ndf = (\n    pl.scan_csv(csv_file, separator=\',\', has_header=False, new_columns=column_names)\n    .filter(pl.col("RFC").is_not_null())    # Filter before processing\n    .collect()              # To process the file in chunks\n)\ndf.head(5)\n'

In [5]:
initial_rows = df.shape[0]
print(f"El archivo tiene {initial_rows} filas.")

# Delete all rows with null values in some columns
df = df.drop_nulls()

# Change column types
df = df.with_columns(
    pl.col([column_names[0],column_names[1]]).cast(pl.String, strict=False)
)

df = df.with_columns(pl.col([column_names[2]]).cast(pl.Int32, strict=False))

El archivo tiene 17 filas.


## Validate RFC format

In [6]:
# Regular expressions
RFC_FISICA_REGEX = re.compile(
    r"^[A-ZÑ&]{4}"
    r"\d{2}(0[1-9]|1[0-2])"
    r"(0[1-9]|[12]\d|3[01])"
    r"[A-Z0-9]{3}$"
)

RFC_MORAL_REGEX = re.compile(
    r"^[A-ZÑ&]{3}"
    r"\d{2}(0[1-9]|1[0-2])"
    r"(0[1-9]|[12]\d|3[01])"
    r"[A-Z0-9]{3}$"
)

# Normalize RFC
df = df.with_columns(
    pl.col("RFC").str.strip_chars().str.strip_chars('.')
    .str.strip_chars().str.to_uppercase()
)

df = (
    df.with_columns(
        pl.when(pl.col("RFC").str.contains(RFC_FISICA_REGEX.pattern if hasattr(RFC_FISICA_REGEX, 'pattern') else RFC_FISICA_REGEX))
        .then(pl.lit("FISICA"))
        .when(pl.col("RFC").str.contains(RFC_MORAL_REGEX.pattern if hasattr(RFC_MORAL_REGEX, 'pattern') else RFC_MORAL_REGEX))
        .then(pl.lit("MORAL"))
        .otherwise(None)
        .alias("PERSONA")
    )
    .filter(pl.col("PERSONA").is_not_null())
)

df.head()

RFC,RAZON,AÑO,PERSONA
str,str,i16,str
"""AAA08091161A""","""APOYANDO A ANGELITOS CON AUTIS…",2025,"""MORAL"""
"""AAA1002249W5""","""LA PASADITA""",2025,"""MORAL"""
"""AAA090924HJ4""","""ARGUELLES, ALVAREZ & ASOCIADOS…",2025,"""MORAL"""
"""AVM090924HJ4""","""ADMINISTRATIVAS APLICACIONES A…",2025,"""MORAL"""
"""AAA1002249W6""","""ADAIR ALONSO ARQUITECTOS SA DE…",2025,"""MORAL"""


This process is applied to "MORAL" persons only.

In [7]:
df = df.filter(pl.col("PERSONA") == "MORAL")

final_rows = df.shape[0]
print(f"El archivo tiene {final_rows} RFCs válidos.")

El archivo tiene 17 RFCs válidos.


## Column "RAZON" preprocessing

In [8]:
# Patterns like S.A., C.V.
norm_rules = [
    (r'\bS\.?\s?A\.? ', "SA "),
    (r'\bS\.?\s?A\.?\b', "SA"),
    (r'\bC\.?\s?V\.?\b', "CV"),
    (r'\bS\.?\s?C\.?\b', "SC"),
    (r'\bS\.?\s?A\.?P\.?\s?I\.?\s?\b', "SAPI "),
    (r'\bA\.?\s?C\.? ', "AC "),
    (r'\bA\.?\s?C\.?\b', "AC"),
]

# Normalize "RAZON"
normalize_text = (
    pl.col(column_names[1])
    .cast(pl.String)
    .str.strip_chars(".,; ")
    .str.replace_all(r"[,;.]", "")
    .str.to_uppercase()
)

for pattern, replacement in norm_rules:
    normalize_text = normalize_text.str.replace_all(pattern, replacement)

normalize_text = (
    normalize_text.str.replace_all(r'[/\\-]', ' ')
    .str.replace_all(r'\s+', ' ')
    .str.strip_chars()
)

df = df.with_columns(
    normalize_text.alias("NOMBRE")
)

df.sample(5)

RFC,RAZON,AÑO,PERSONA,NOMBRE
str,str,i16,str,str
"""BBB08091161A""","""BBVA BANCOMER""",2025,"""MORAL""","""BBVA BANCOMER"""
"""AAA1002249W8""","""Waltmart de Mexico""",2025,"""MORAL""","""WALTMART DE MEXICO"""
"""AAA1002249W6""","""ADAIR ALONSO ARQUITECTOS SA DE…",2025,"""MORAL""","""ADAIR ALONSO ARQUITECTOS SA DE…"
"""FEM1002249W5""","""FEMSA COCA-COLA""",2025,"""MORAL""","""FEMSA COCA COLA"""
"""AVM090924HJ4""","""ADMINISTRATIVAS APLICACIONES A…",2025,"""MORAL""","""ADMINISTRATIVAS APLICACIONES A…"


## Group by words simility with SLM (Small Language Model)

In [9]:
# Download the model
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 25.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
# Load the model
nlp = spacy.load("es_core_news_sm", disable=["ner", "parser"])

# Obtain unique names in column "NOMBRE"
unique_nombres = df.select("NOMBRE").unique()

def lemmatize_pipe(s: pl.Series) -> pl.Series:
    # Batch processing only on the unique
    return pl.Series([
        " ".join([t.lemma_ for t in doc]).upper()
        for doc in nlp.pipe(s.str.to_lowercase(), batch_size=500)
    ])

# Create a lemmatization "map" (small DataFrame)
lemmas_map = unique_nombres.with_columns(
    pl.col("NOMBRE").map_batches(lemmatize_pipe).alias("LEMMA_SPA")
)

# Join the map to the original DataFrame
# to instantly assign the correct lemma to each original row
df = df.join(lemmas_map, on="NOMBRE", how="left")

df.select(["NOMBRE", "LEMMA_SPA"]).head(5)

NOMBRE,LEMMA_SPA
str,str
"""APOYANDO A ANGELITOS CON AUTIS…","""APOYAR A ANGELITO CON AUTISMO …"
"""LA PASADITA""","""EL PASADITA"""
"""ARGUELLES ALVAREZ & ASOCIADOS …","""ARGUELL ALVAREZ & ASOCIADOS SA…"
"""ADMINISTRATIVAS APLICACIONES A…","""ADMINISTRATIVO APLICACIÓN AVM …"
"""ADAIR ALONSO ARQUITECTOS SA DE…","""ADAIR ALONSO ARQUITECTOS SA DE…"


In [11]:
# Sort by 'RFC'
df = df.sort("RFC", descending=False)

In [23]:
"""
def group_by_rfc_and_similarity_polars(df: pl.DataFrame, umbral=75) -> pl.DataFrame:
    '''Reorder df by column "LEMMA_SPA" according to rfc and words simility'''
    data = df.to_dicts()
    n = len(data)
    group_assignment = [-1] * n
    grupo_actual = 0

    for i in range(n):
        if group_assignment[i] == -1:
            group_assignment[i] = grupo_actual
            rfc_base = data[i]['RFC']
            nombre_base = data[i]['LEMMA_SPA']

            # Compare with those who do not have a group only
            for j in range(i + 1, n):
                if group_assignment[j] == -1:
                    # If RFC is the same, then group
                    if rfc_base == data[j]['RFC']:
                        group_assignment[j] = grupo_actual
                    else:
                        # Ignore words order (token_sort_ratio)
                        score = fuzz.token_sort_ratio(nombre_base, data[j]['LEMMA_SPA'])
                        if score >= umbral:
                            group_assignment[j] = grupo_actual

            grupo_actual += 1

    # Order by "group" and drop this column
    return (
        df.with_columns(pl.Series("group", group_assignment))
        .sort("group")
        .drop("group")
    )

# Group by RFC and text similarity
df_ordered = group_by_rfc_and_similarity_polars(df, 75)
"""

'\ndef group_by_rfc_and_similarity_polars(df: pl.DataFrame, umbral=75) -> pl.DataFrame:\n    \'\'\'Reorder df by column "LEMMA_SPA" according to rfc and words simility\'\'\'\n    data = df.to_dicts()\n    n = len(data)\n    group_assignment = [-1] * n\n    grupo_actual = 0\n\n    for i in range(n):\n        if group_assignment[i] == -1:\n            group_assignment[i] = grupo_actual\n            rfc_base = data[i][\'RFC\']\n            nombre_base = data[i][\'LEMMA_SPA\']\n\n            # Compare with those who do not have a group only\n            for j in range(i + 1, n):\n                if group_assignment[j] == -1:\n                    # If RFC is the same, then group\n                    if rfc_base == data[j][\'RFC\']:\n                        group_assignment[j] = grupo_actual\n                    else:\n                        # Ignore words order (token_sort_ratio)\n                        score = fuzz.token_sort_ratio(nombre_base, data[j][\'LEMMA_SPA\'])\n                  

In [13]:
def group_by_rfc_and_similarity(df: pl.DataFrame, umbral=75) -> pl.DataFrame:
    # Group by RFC
    # Create an ID for unique RFC
    df = df.with_columns(
        pl.col("RFC").rank("dense").alias("group_id")
    )

    # Dimensionality Reduction for Fuzzy Matching
    # Compare UNIQUE names that do not share the same RFC
    unique_pairs = (
        df.select(["RFC", "LEMMA_SPA", "group_id"])
        .unique(subset=["RFC"])         # One representative per RFC
    )

    # Fuzzy Matching on the Reduced Set
    # Even for 1M rows, the unique set can still be large.
    # Comparison logic is applied, but on 'unique_pairs'
    data = unique_pairs.to_dicts()
    n = len(data)
    remap_groups = {}         # Dictionary for merging similar groups

    # We limit the search to one window or use a prefix technique
    # to avoid comparing 1M vs 1M.
    for i in range(min(n, 10000)):
        base_name = data[i]['LEMMA_SPA']
        base_group = data[i]['group_id']

        for j in range(i + 1, min(n, i + 500)):     # Sliding window (Blocking)
            if fuzz.token_sort_ratio(base_name, data[j]['LEMMA_SPA']) >= umbral:
                remap_groups[data[j]['group_id']] = base_group

    # Apply the remapping to the original DataFrame
    df = df.with_columns(
        pl.col("group_id").replace(remap_groups)
    )

    return df.sort("group_id").drop("group_id")

In [14]:
# Group by RFC and text similarity
df_ordered = group_by_rfc_and_similarity(df,75)

df_ordered

RFC,RAZON,AÑO,PERSONA,NOMBRE,LEMMA_SPA
str,str,i16,str,str,str
"""AAA08091161A""","""APOYANDO A ANGELITOS CON AUTIS…",2025,"""MORAL""","""APOYANDO A ANGELITOS CON AUTIS…","""APOYAR A ANGELITO CON AUTISMO …"
"""AAA1002249W9""","""apoyando angelitos con autismo…",2025,"""MORAL""","""APOYANDO ANGELITOS CON AUTISMO…","""APOYAR ANGELITO CON AUTISMO AC"""
"""AAA090924HJ4""","""ARGUELLES, ALVAREZ & ASOCIADOS…",2025,"""MORAL""","""ARGUELLES ALVAREZ & ASOCIADOS …","""ARGUELL ALVAREZ & ASOCIADOS SA…"
"""AAA1002249W1""","""MC SA DE CV""",2025,"""MORAL""","""MC SA DE CV""","""MC SA DE CV"""
"""AAA1002249W2""","""S.A. DE C.V. MC""",2025,"""MORAL""","""SA DE CV MC""","""SA DE CV MC"""
…,…,…,…,…,…
"""BBB08091161A""","""BBVA BANCOMER""",2025,"""MORAL""","""BBVA BANCOMER""","""BBVA BANCOMER"""
"""BBB08091161A""","""BBVA BANCOMER""",2025,"""MORAL""","""BBVA BANCOMER""","""BBVA BANCOMER"""
"""BBV08091161A""","""BANCOMER BBVA S.A.""",2025,"""MORAL""","""BANCOMER BBVA SA""","""BANCOMER BBVA SA"""


In [15]:
# Reorder columns
df1 = df_ordered.select(['RFC', 'NOMBRE', 'PERSONA', 'AÑO', 'LEMMA_SPA', 'RAZON'])

# Save DataFrame
file_name = f"NuevosRFC"
# file_name = f"NuevosRFC_{csv_file.split('.')[0]}"
df1.write_csv(f"{file_name}.csv", separator=",", include_header=True)

### Concat 2 DataFrames (optional)

For simplicity, the same DataFrame will be concated to itself.

In [17]:
df2 = df1

# Add year
df2 = df2.with_columns(pl.lit(year+1).alias("AÑO"))
df2

RFC,NOMBRE,PERSONA,AÑO,LEMMA_SPA,RAZON
str,str,str,i32,str,str
"""AAA08091161A""","""APOYANDO A ANGELITOS CON AUTIS…","""MORAL""",2026,"""APOYAR A ANGELITO CON AUTISMO …","""APOYANDO A ANGELITOS CON AUTIS…"
"""AAA1002249W9""","""APOYANDO ANGELITOS CON AUTISMO…","""MORAL""",2026,"""APOYAR ANGELITO CON AUTISMO AC""","""apoyando angelitos con autismo…"
"""AAA090924HJ4""","""ARGUELLES ALVAREZ & ASOCIADOS …","""MORAL""",2026,"""ARGUELL ALVAREZ & ASOCIADOS SA…","""ARGUELLES, ALVAREZ & ASOCIADOS…"
"""AAA1002249W1""","""MC SA DE CV""","""MORAL""",2026,"""MC SA DE CV""","""MC SA DE CV"""
"""AAA1002249W2""","""SA DE CV MC""","""MORAL""",2026,"""SA DE CV MC""","""S.A. DE C.V. MC"""
…,…,…,…,…,…
"""BBB08091161A""","""BBVA BANCOMER""","""MORAL""",2026,"""BBVA BANCOMER""","""BBVA BANCOMER"""
"""BBB08091161A""","""BBVA BANCOMER""","""MORAL""",2026,"""BBVA BANCOMER""","""BBVA BANCOMER"""
"""BBV08091161A""","""BANCOMER BBVA SA""","""MORAL""",2026,"""BANCOMER BBVA SA""","""BANCOMER BBVA S.A."""


In [22]:
# Concat DataFrames
df_final = pl.concat([df1, df2], how="vertical")
df_final.sample(5)

RFC,NOMBRE,PERSONA,AÑO,LEMMA_SPA,RAZON
str,str,str,i32,str,str
"""AAA090924HJ4""","""ARGUELLES ALVAREZ & ASOCIADOS …","""MORAL""",2025,"""ARGUELL ALVAREZ & ASOCIADOS SA…","""ARGUELLES, ALVAREZ & ASOCIADOS…"
"""BBB08091161A""","""BBVA BANCOMER""","""MORAL""",2025,"""BBVA BANCOMER""","""BBVA BANCOMER"""
"""AAA1002249W6""","""ADAIR ALONSO ARQUITECTOS SA DE…","""MORAL""",2026,"""ADAIR ALONSO ARQUITECTOS SA DE…","""ADAIR ALONSO ARQUITECTOS SA DE…"
"""FEM1002249W5""","""COCA COLA FEMSA CV""","""MORAL""",2025,"""COCAR COLA FEMSIR CV""","""COCA COLA FEMSA CV"""
"""BBB08091161A""","""BBVA BANCOMER""","""MORAL""",2025,"""BBVA BANCOMER""","""BBVA BANCOMER"""


In [25]:
# Sort by 'RFC'
df_final = df_final.sort("RFC", descending=False)

# Group by RFC and text similarity
df_ordered = group_by_rfc_and_similarity(df_final,75)

# Reorder columns
df = df_ordered.select(['RFC', 'NOMBRE', 'PERSONA', 'AÑO', 'LEMMA_SPA', 'RAZON'])

# Save DataFrame
file_name = f"RFC_final"
df.write_csv(f"{file_name}.csv", separator=",", include_header=True)